In [6]:
import tensorflow as tf
import os
import datetime as dtime
import time

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#VGG
conv_layers = [
    tf.keras.layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(64,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
    tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(128,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
    tf.keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(256,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
    tf.keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    
    tf.keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(512,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same')    
]

conv_net = tf.keras.Sequential(conv_layers)

# x = tf.random.normal([4,32,32,3])
# print(conv_net(x).shape)

fc_net = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(100)
])

conv_net.build(input_shape=[None,32,32,3])
fc_net.build(input_shape=[None,512])
conv_net.summary()
fc_net.summary()
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255
    y = tf.cast(y,dtype=tf.int32)
    return x,y

#attention
def attention(inputs):
    input_channels = int(inputs.shape[-1])
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(int(input_channels/2))(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dense(input_channels)(x)
    x = tf.keras.layers.Activation('softmax')(x)
    x = tf.reshape(x,[100,1,1,input_channels])
    x = tf.multiply(x,inputs)
    return x
    
(x,y),(x_test,y_test) = tf.keras.datasets.cifar100.load_data()
y = tf.squeeze(y,axis=1)
y_test = tf.squeeze(y_test,axis=1)

train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(preprocess).batch(100)

test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.map(preprocess).shuffle(1000).batch(100)

learn_rate = 1e-4
epoch_num = 100
optimizer = tf.keras.optimizers.Adam(lr = learn_rate)

current_time = dtime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = '../logs/'+current_time
summary_writer = tf.summary.create_file_writer(log_dir)

#两个网络参数拼接
variables = conv_net.trainable_variables+fc_net.trainable_variables

pre_loss,cur_loss = 0.,0.
start = dtime.datetime.now()
num_count = 0
for epoch in range(epoch_num):
    for step,(x,y) in enumerate(train_db):
        num_count+=1
        with tf.GradientTape() as tape:
            out = conv_net(x)
            out = tf.reshape(out,[-1,512])
            logits = fc_net(out)
            y_onehot = tf.one_hot(y,depth=100)
            
            loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True))
            pre_loss = cur_loss
            cur_loss = loss
        
        grads = tape.gradient(loss,variables)
        optimizer.apply_gradients(zip(grads,variables))
        
        if step % 100 == 0:
            with summary_writer.as_default():
                tf.summary.scalar('train_loss',float(loss),step=num_count)
            print('epoch:',epoch,'step:',step,'loss:',loss)
    
    optimizer.lr = learn_rate*(epoch_num-epoch)/epoch_num
    if epoch>20 and cur_loss-pre_loss>0.1:
        break;
        
    total_num = 0
    total_correct = 0
    for x,y in test_db:
        out = conv_net(x)
        out = tf.reshape(out,[-1,512])
        logits = fc_net(out)
        prob = tf.nn.softmax(logits,axis=1)
        pred = tf.cast(tf.argmax(prob,axis=1),dtype=tf.int32)
        correct = tf.reduce_sum(tf.cast(tf.equal(pred,y),dtype=tf.int32))
        
        total_num +=x.shape[0]
        total_correct +=int(correct)
    acc = total_correct/total_num
    with summary_writer.as_default():
        tf.summary.scalar('test_acc',float(acc),step = epoch)
    
    if acc > 0.9:
        break;
    print('epoch:',epoch,'acc:',acc)
    
end = dtime.datetime.now()
print(end - start)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_20 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
batch_normalization_21 (Batc (None, 16, 16, 128)      

KeyboardInterrupt: 

In [ ]:
# import tensorflow as tf
# import matplotlib.pyplot as plt
# import os

# os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# (train_x,train_y),(test_x,test_y) = tf.keras.datasets.boston_housing.load_data()
# # print(train_x[:,2].shape)

# plt.xlabel('price')
# plt.ylabel('4')
# plt.scatter(train_x[:,4],train_y)


In [ ]:
#Test

# import tensorflow as tf

# a = tf.constant([[[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]],[[1,2,3],[4,5,6],[7,8,9]]]])
# b = tf.constant([[[2,4,8]]])

# print(a.shape)
# print(b.shape)
# n= a*b
# m = tf.multiply(a,b)
# print(n)
# print(m)